In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

See /nfs/home/genovese/thesis-wildfire-genovese/outputs/EDA_ProfileReport.html for the EDA, here the consequent preprocessing

In [2]:
data = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/database/model_input/table_data_input.csv')

### 1. drop duplicates and constant variables

In [ ]:
data.drop_duplicates(inplace=True)
data.drop(columns=[col for col in data.columns if (data[col].nunique == 1)],inplace=True)
data.dropna(axis=1, how='all', inplace=True)
data.drop(index=data.index[0], inplace=True)

### 2. Check for almost constant variables

In [ ]:
for c in data.columns:
    vals = data[c].value_counts().sort_values(ascending = False)
    if vals.iloc[1:].sum() <= 50:
        print(f'Column {c} has value counts: ', vals)

In [13]:
data['TOTALE_VEGETAZIONE'] = data.loc[:, [col for col in data.columns if 'VEGETAZIONE_TIPO' in col]].sum(axis=1)
data['TOTALE_STRADE'] = data.loc[:, [col for col in data.columns if 'RTP' in col]].sum(axis=1)

In [14]:
to_drop = []

for c in data.columns:
    vals = data[c].value_counts().sort_values(ascending = False)
    if vals.iloc[1:].sum() <= 10:
        if np.max(list(vals.index)) < 0.1:
            to_drop.append(c)

In [16]:
data.drop(columns=to_drop, inplace=True)

### 3. Look for too high correlations

In [17]:
def find_high_corr_pairs(df, threshold=0.9):
    corr = df.corr()  # compute correlation matrix
    # Extract only upper triangle (exclude duplicates and self-correlation)
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    
    # Stack into long format and filter
    high_corr = upper.stack().loc[lambda x: x.abs() > threshold]
    # Result is a Series with MultiIndex (col1, col2) and value = correlation

    return high_corr

In [19]:
high_corr = find_high_corr_pairs(data.drop(columns='ignition_point (epsg=3857)'), threshold=0.95)
for s in high_corr.index:
    print(s)

('analfabeti', 'alfabeti privi di titolo di studio')
('analfabeti', 'licenza di scuola elementare')
('analfabeti', 'licenza di scuola media inferiore o di avviamento professionale')
('analfabeti', 'diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) compresi IFTS')
('analfabeti', 'diploma di tecnico superiore ITS o titolo di studio terziario di primo livello')
('analfabeti', 'titolo di studio terziario di secondo livello e dottorato di ricerca')
('analfabeti', 'titolo di studio terziario di secondo livello')
('analfabeti', 'dottorato di ricerca/diploma accademico di formazione alla ricerca')
('analfabeti', 'totale_istruzione')
('analfabeti', '9_24_nessun titolo di studio')
('analfabeti', '9_24_licenza di scuola elementare')
('analfabeti', '9_24_licenza di scuola media inferiore o di avviamento professionale')
('analfabeti', '9_24_diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) compresi IFTS')
('anal

In [20]:
save_clean_data(data.drop(columns='ignition_point (epsg=3857)'), 'processed_not_imputed', '/nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/database/cache')

File saved to: /nfs/home/genovese/thesis-wildfire-genovese/rekriging_target/database/cache/processed_not_imputed


### 4. NA imputing
see knn_imputer.py, to do inside each kfold